# Notebook

In [116]:
import math
import cmath
import yaml

## Book Keeping

In [117]:
MODEL = "PE1R_112_M4"
VERBOSITY = 1

In [118]:
RATED_SPEED = 1000
RATED_FREQUENCY = 50
RATED_POWER = 90e3
VOLTAGE = 361
POWER_FACTOR = 0.97
RATED_CURRENT = 158
PHASE_RESISTANCE = 0.012
MAGNETISING_INDUCTANCE = 0.79e-3
LEAKAGE_INDUCTANCE = 0.24e-3
MOTOR_MOMENT_OF_INERTIA = 2.42

In [119]:
P = (120 * RATED_FREQUENCY) / RATED_SPEED
we = RATED_FREQUENCY * (2 * math.pi)
ws = RATED_SPEED * (math.pi / 30)
wr = ws

if VERBOSITY:
    print("P :: {:.4f}".format(P))
    print("we :: {:.4f}".format(we))
    print("ws :: {:.4f}".format(ws))
    print("wr :: {:.4f}".format(wr))

P :: 6.0000
we :: 314.1593
ws :: 104.7198
wr :: 104.7198


In [120]:
Ld = LEAKAGE_INDUCTANCE + (3 / 2) * MAGNETISING_INDUCTANCE
Lq = LEAKAGE_INDUCTANCE + (3 / 2) * MAGNETISING_INDUCTANCE
Ls = LEAKAGE_INDUCTANCE + (3 / 2) * MAGNETISING_INDUCTANCE

if VERBOSITY:
    print("Ld :: {:.4f}m".format(Ld * 1000))
    print("Lq :: {:.4f}m".format(Lq * 1000))
    print("Ls :: {:.4f}m".format(Ls * 1000))

Ld :: 1.4250m
Lq :: 1.4250m
Ls :: 1.4250m


In [121]:
Va = VOLTAGE / math.sqrt(3)
Ia = cmath.rect(RATED_CURRENT, -math.acos(POWER_FACTOR))
Eaf = Va - Ia * complex(PHASE_RESISTANCE, Ls * we)
Vpk_krpm = math.sqrt(2) * ((math.sqrt(3) * abs(Eaf)) / (RATED_SPEED / 1000))

if VERBOSITY:
    print("Va :: {:.4f}".format(Va))
    print("Ia :: {:.4f}".format(Ia))
    print("Eaf :: {:.4f}".format(Eaf))
    print("Vpk_krpm :: {:.4f}".format(Vpk_krpm))

Va :: 208.4234
Ia :: 153.2600-38.4106j
Eaf :: 189.3888-68.1500j
Vpk_krpm :: 493.0266


In [122]:
Pe = 3 * (Eaf * Ia.conjugate()).real
Br = (Pe - RATED_POWER) / ws**2
tau_shaft = MOTOR_MOMENT_OF_INERTIA / Br

if VERBOSITY:
    print("Pe :: {:.4f}".format(Pe))
    print("Br :: {:.4f}m".format(Br * 1000))
    print("tau_shaft :: {:.4f}".format(tau_shaft))

Pe :: 94930.2285
Br :: 449.5829m
tau_shaft :: 5.3828


In [123]:
lambda_fm = (math.sqrt(2) * abs(Eaf)) / ((P / 2) * wr)

if VERBOSITY:
    print("lambda_fm :: {:.4f}".format(lambda_fm))

lambda_fm :: 0.9061


In [124]:
Ke = (P / 2) * lambda_fm
Kt = (3 / 2) * (P / 2) * lambda_fm

if VERBOSITY:
    print("Ke :: {:.4f}".format(Ke))
    print("Kt :: {:.4f}".format(Kt))

Ke :: 2.7182
Kt :: 4.0773


In [125]:
with open("../reports/" + MODEL + "_parameters.yaml", "w") as file:
    documents = yaml.dump({
            "provided_parameters": {
                "Rs": PHASE_RESISTANCE,
                "Jr": MOTOR_MOMENT_OF_INERTIA,
            }, "estimated_parameters": {
                "Ld": Ld,
                "Lq": Lq,
                "Vpk_krpm": Vpk_krpm,
                "Ke": Ke,
                "Kt": Kt,
                "P": P,
                "Br": Br,
                "tau_shaft": tau_shaft,
            }
        }, 
        file
    )